In [ ]:
import time
import os
import pdfplumber
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

# Set the path to the download directory
download_dir = "/Users/kitchphil/Desktop/brittanyferriesdump"

# Setup Firefox options with custom download settings
def setup_firefox_driver():
    firefox_options = Options()
    firefox_options.set_preference("browser.download.folderList", 2)  # Use custom location (2)
    firefox_options.set_preference("browser.download.dir", download_dir)  # Set the download directory
    firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")  # Auto-download PDFs
    firefox_options.set_preference("pdfjs.disabled", True)  # Disable the PDF viewer to force download

    # Setup the Firefox WebDriver with the options
    service = Service(executable_path='/Users/kitchphil/Downloads/geckodriver')  # Adjust the path to geckodriver
    return webdriver.Firefox(service=service, options=firefox_options)

# Function to handle cookie consent popup and remove the overlay
def accept_cookies_and_remove_overlay(driver):
    try:
        # Accept the cookies first
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
        )
        accept_button.click()
        print("Cookies accepted.")
        
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'onetrust-pc-dark-filter'))
        )
        print("Cookie overlay has disappeared.")
        
        # Use JavaScript to ensure any remaining overlay is removed
        driver.execute_script(
            "document.querySelector('.onetrust-pc-dark-filter').style.display = 'none';"
        )
        print("Overlay removed using JavaScript.")
        
    except Exception as e:
        print("No cookie consent popup found or another issue occurred:", e)

# Function to click the chevron icon (dropdown) with retry mechanism
def click_chevron_icon(driver):
    try:
        # Try to click the chevron icon with an explicit wait
        chevron_icon = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//mat-icon[@data-mat-icon-name="baicon-chevron-down"]'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", chevron_icon)
        chevron_icon.click()
        print("Chevron icon clicked.")
    except Exception as e:
        print(f"Error occurred while clicking the chevron icon: {e}. Retrying...")
        # Retry clicking the chevron after ensuring overlay is removed
        accept_cookies_and_remove_overlay(driver)
        chevron_icon = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//mat-icon[@data-mat-icon-name="baicon-chevron-down"]'))
        )
        chevron_icon.click()
        print("Chevron icon clicked on retry.")

# Function to select the 5th option using JavaScript
def select_5th_option_with_js(driver):
    try:
        # Wait for the overlay to appear
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "cdk-overlay-container"))
        )
        print("Overlay appeared.")
        
        # Use JavaScript to select the 5th option
        driver.execute_script(
            "document.querySelector('#cdk-overlay-0 > div.ng-tns-c3113121375-6.bf-adaptive-field-panel-wrap.undefined.is-desktop.ng-trigger.ng-trigger-transformPanel.ng-star-inserted > bf-reactive-adaptive-field > div > div > mat-selection-list > mat-list-option:nth-child(5)').click();"
        )
        print("5th option selected via JavaScript.")
    except Exception as e:
        print(f"Error selecting the 5th option via JavaScript: {e}")

# Function to select the 6th option using JavaScript
def select_6th_option_with_js(driver):
    try:
        # Wait for the overlay to appear
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "cdk-overlay-container"))
        )
        print("Overlay appeared.")
        
        # Use JavaScript to select the 6th option
        driver.execute_script(
            "document.querySelector('#cdk-overlay-0 > div.ng-tns-c3113121375-6.bf-adaptive-field-panel-wrap.undefined.is-desktop.ng-trigger.ng-trigger-transformPanel.ng-star-inserted > bf-reactive-adaptive-field > div > div > mat-selection-list > mat-list-option:nth-child(6)').click();"
        )
        print("6th option selected via JavaScript.")
    except Exception as e:
        print(f"Error selecting the 6th option via JavaScript: {e}")

# Function to click the button by XPath
def click_button_by_xpath(driver):
    try:
        button_xpath = "/html/body/dd-root/mat-sidenav-container/mat-sidenav-content/main/dd-cabin-availability/mat-tab-group/div/mat-tab-body[1]/div/div/div/dd-cabin-availability-pdf-panel/section/div[2]/div/div/form/button/span[2]"
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, button_xpath))
        )
        button.click()
        print("Button clicked successfully.")
    except Exception as e:
        print(f"Error clicking the button: {e}")

# Function to click the download button in the PDF viewer
def click_download_button_in_pdf_viewer(driver):
    try:
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='mat-tab-content-0-0']/div/div/div/dd-cabin-availability-pdf-panel/section/iframe"))
        )
        driver.switch_to.frame(iframe)
        
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='download']"))
        )
        download_button.click()
        print("Download button clicked.")
        
        driver.switch_to.default_content()
    except Exception as e:
        print(f"Error clicking the download button: {e}")

# Phase 1: Select the 5th option and download PDF
driver = setup_firefox_driver()
driver.get('https://www.brittany-ferries.co.uk/cabin-availability?availabilityType=CABIN')

# Accept cookies, remove overlay, and handle page
accept_cookies_and_remove_overlay(driver)
click_chevron_icon(driver)
select_5th_option_with_js(driver)  # Select the 5th option
click_button_by_xpath(driver)
click_download_button_in_pdf_viewer(driver)

# Close WebDriver after the first download
driver.quit()

# Phase 2: Reopen WebDriver, select the 6th option, and download PDF
driver = setup_firefox_driver()
driver.get('https://www.brittany-ferries.co.uk/cabin-availability?availabilityType=CABIN')

# Accept cookies, remove overlay, and handle page again
accept_cookies_and_remove_overlay(driver)
click_chevron_icon(driver)  # Retry mechanism for chevron click
select_6th_option_with_js(driver)  # Select the 6th option using JavaScript
click_button_by_xpath(driver)
click_download_button_in_pdf_viewer(driver)

# Close WebDriver after the second download
driver.quit()

# Extract data from downloaded PDFs using pdfplumber
def extract_tables_with_pdfplumber(directory_path):
    tables_data = {}
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            
            # Open PDF with pdfplumber
            with pdfplumber.open(file_path) as pdf:
                pdf_tables = []
                
                # Iterate through each page in the PDF
                for page_num, page in enumerate(pdf.pages):
                    tables = page.extract_tables()
                    pdf_tables.extend(tables)
                tables_data[filename] = pdf_tables
    
    return tables_data

# Example: Extract and process PDF tables
pdf_tables = extract_tables_with_pdfplumber(download_dir)
data = []
for pdf, tables in pdf_tables.items():
    for table in tables:
        for row in table:
            if len(row) == 9 and row[0]:
                data.append(row)

# Process extracted data into a DataFrame
columns = ['Ship', 'Route', 'Date & Time', 'N1', 'N2', 'N3', 'N4', 'Pets1', 'Pets2']
df = pd.DataFrame(data, columns=columns)
df['Pets'] = df['Pets1'].astype(int) + df['Pets2'].astype(int)
dffilt = df[['Ship', 'Route', 'Date & Time', 'Pets']]
dffilt = dffilt[dffilt['Route'].str[0] != 'P']
dffilt = dffilt[(dffilt['Date & Time'].str[3:8] == '11/24') | (dffilt['Date & Time'].str[3:8] == '12/24')]

# Check if there are any pets available and take action
if len(dffilt[dffilt['Pets'] > 0]) > 0:
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    
    def send_email_alert(subject, message, recipients):
        from_email = "philip.kitchener1989@gmail.com"  # Your email address
        password = "qlfwzcrviuczfujj"  # Your email password
    
        # Create the email message
        msg = MIMEMultipart()
        msg['From'] = from_email
        msg['To'] = ", ".join(recipients)  # Join the recipients' emails with commas
        msg['Subject'] = subject
    
        # Add the message body
        msg.attach(MIMEText(message, 'plain'))
    
        # Set up the server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Start TLS encryption
        server.login(from_email, password)  # Log in to your email account
    
        # Send the email to all recipients
        server.sendmail(from_email, recipients, msg.as_string())
    
        # Close the server connection
        server.quit()
    
        print(f"Email sent to: {', '.join(recipients)}")
    
    # Example Usage
    subject = "Pets Available Notification"
    message = "Pets are available on the following routes and dates."
    
    # List of email recipients
    recipients = ["beaquel13@gmail.com", "philip.kitchener1989@gmail.com"]  # Add as many emails as you need
    
    # Send the email
    send_email_alert(subject, message, recipients)

else:
    print('none found at this time')


In [117]:
import os
import pdfplumber
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

# Set the path to the download directory
download_dir = "/Users/kitchphil/Desktop/brittanyferriesdump"

# Setup Firefox options with custom download settings
def setup_firefox_driver():
    firefox_options = Options()
    firefox_options.set_preference("browser.download.folderList", 2)  # Use custom location (2)
    firefox_options.set_preference("browser.download.dir", download_dir)  # Set the download directory
    firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")  # Auto-download PDFs
    firefox_options.set_preference("pdfjs.disabled", True)  # Disable the PDF viewer to force download

    # Setup the Firefox WebDriver with the options
    service = Service(executable_path='/Users/kitchphil/Downloads/geckodriver')  # Adjust the path to geckodriver
    return webdriver.Firefox(service=service, options=firefox_options)

# Function to handle cookie consent popup and remove the overlay
def accept_cookies_and_remove_overlay(driver):
    try:
        # Accept the cookies first
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
        )
        accept_button.click()
        print("Cookies accepted.")
        
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'onetrust-pc-dark-filter'))
        )
        print("Cookie overlay has disappeared.")
        
        # Use JavaScript to ensure any remaining overlay is removed
        driver.execute_script(
            "document.querySelector('.onetrust-pc-dark-filter').style.display = 'none';"
        )
        print("Overlay removed using JavaScript.")
        
    except Exception as e:
        print("No cookie consent popup found or another issue occurred:", e)

# Function to click the chevron icon (dropdown) with retry mechanism
def click_chevron_icon(driver):
    try:
        # Try to click the chevron icon with an explicit wait
        chevron_icon = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//mat-icon[@data-mat-icon-name="baicon-chevron-down"]'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", chevron_icon)
        chevron_icon.click()
        print("Chevron icon clicked.")
    except Exception as e:
        print(f"Error occurred while clicking the chevron icon: {e}. Retrying...")
        # Retry clicking the chevron after ensuring overlay is removed
        accept_cookies_and_remove_overlay(driver)
        chevron_icon = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//mat-icon[@data-mat-icon-name="baicon-chevron-down"]'))
        )
        chevron_icon.click()
        print("Chevron icon clicked on retry.")

# Function to select the 5th option using JavaScript
def select_5th_option_with_js(driver):
    try:
        # Wait for the overlay to appear
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "cdk-overlay-container"))
        )
        print("Overlay appeared.")
        
        # Use JavaScript to select the 5th option
        driver.execute_script(
            "document.querySelector('#cdk-overlay-0 > div.ng-tns-c3113121375-6.bf-adaptive-field-panel-wrap.undefined.is-desktop.ng-trigger.ng-trigger-transformPanel.ng-star-inserted > bf-reactive-adaptive-field > div > div > mat-selection-list > mat-list-option:nth-child(5)').click();"
        )
        print("5th option selected via JavaScript.")
    except Exception as e:
        print(f"Error selecting the 5th option via JavaScript: {e}")

# Function to select the 6th option using JavaScript
def select_6th_option_with_js(driver):
    try:
        # Wait for the overlay to appear
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "cdk-overlay-container"))
        )
        print("Overlay appeared.")
        
        # Use JavaScript to select the 6th option
        driver.execute_script(
            "document.querySelector('#cdk-overlay-0 > div.ng-tns-c3113121375-6.bf-adaptive-field-panel-wrap.undefined.is-desktop.ng-trigger.ng-trigger-transformPanel.ng-star-inserted > bf-reactive-adaptive-field > div > div > mat-selection-list > mat-list-option:nth-child(6)').click();"
        )
        print("6th option selected via JavaScript.")
    except Exception as e:
        print(f"Error selecting the 6th option via JavaScript: {e}")

# Function to click the button by XPath
def click_button_by_xpath(driver):
    try:
        button_xpath = "/html/body/dd-root/mat-sidenav-container/mat-sidenav-content/main/dd-cabin-availability/mat-tab-group/div/mat-tab-body[1]/div/div/div/dd-cabin-availability-pdf-panel/section/div[2]/div/div/form/button/span[2]"
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, button_xpath))
        )
        button.click()
        print("Button clicked successfully.")
    except Exception as e:
        print(f"Error clicking the button: {e}")

# Function to click the download button in the PDF viewer
def click_download_button_in_pdf_viewer(driver):
    try:
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='mat-tab-content-0-0']/div/div/div/dd-cabin-availability-pdf-panel/section/iframe"))
        )
        driver.switch_to.frame(iframe)
        
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='download']"))
        )
        download_button.click()
        print("Download button clicked.")
        
        driver.switch_to.default_content()
    except Exception as e:
        print(f"Error clicking the download button: {e}")

# Phase 1: Select the 5th option and download PDF
driver = setup_firefox_driver()
driver.get('https://www.brittany-ferries.co.uk/cabin-availability?availabilityType=CABIN')

# Accept cookies, remove overlay, and handle page
accept_cookies_and_remove_overlay(driver)
click_chevron_icon(driver)
select_5th_option_with_js(driver)  # Select the 5th option
click_button_by_xpath(driver)
click_download_button_in_pdf_viewer(driver)

# Close WebDriver after the first download
driver.quit()

# Phase 2: Reopen WebDriver, select the 6th option, and download PDF
driver = setup_firefox_driver()
driver.get('https://www.brittany-ferries.co.uk/cabin-availability?availabilityType=CABIN')

# Accept cookies, remove overlay, and handle page again
accept_cookies_and_remove_overlay(driver)
click_chevron_icon(driver)  # Retry mechanism for chevron click
select_6th_option_with_js(driver)  # Select the 6th option using JavaScript
click_button_by_xpath(driver)
click_download_button_in_pdf_viewer(driver)

# Close WebDriver after the second download
driver.quit()

# Extract data from downloaded PDFs using pdfplumber
def extract_tables_with_pdfplumber(directory_path):
    tables_data = {}
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            
            # Open PDF with pdfplumber
            with pdfplumber.open(file_path) as pdf:
                pdf_tables = []
                
                # Iterate through each page in the PDF
                for page_num, page in enumerate(pdf.pages):
                    tables = page.extract_tables()
                    pdf_tables.extend(tables)
                tables_data[filename] = pdf_tables
    
    return tables_data

# Function to delete all PDFs in the directory
def delete_pdfs_from_folder(directory_path):
    try:
        for filename in os.listdir(directory_path):
            if filename.endswith(".pdf"):
                file_path = os.path.join(directory_path, filename)
                os.remove(file_path)
                print(f"Deleted: {file_path}")
    except Exception as e:
        print(f"Error deleting files: {e}")

# Example: Extract and process PDF tables
pdf_tables = extract_tables_with_pdfplumber(download_dir)
data = []
for pdf, tables in pdf_tables.items():
    for table in tables:
        for row in table:
            if len(row) == 9 and row[0]:
                data.append(row)

# Process extracted data into a DataFrame
columns = ['Ship', 'Route', 'Date & Time', 'N1', 'N2', 'N3', 'N4', 'Pets1', 'Pets2']
df = pd.DataFrame(data, columns=columns)
df['Pets'] = df['Pets1'].astype(int) + df['Pets2'].astype(int)
dffilt = df[['Ship', 'Route', 'Date & Time', 'Pets']]
dffilt = dffilt[dffilt['Route'].str[0] != 'P']
dffilt = dffilt[(dffilt['Date & Time'].str[3:8] == '11/24') | (dffilt['Date & Time'].str[3:8] == '12/24')]

# Check if there are any pets available and take action
if len(dffilt[dffilt['Pets'] > 0]) > 0:
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    
    def send_email_alert(subject, message, recipients):
        from_email = "philip.kitchener1989@gmail.com"  # Your email address
        password = "qlfwzcrviuczfujj"  # Your email password
    
        # Create the email message
        msg = MIMEMultipart()
        msg['From'] = from_email
        msg['To'] = ", ".join(recipients)  # Join the recipients' emails with commas
        msg['Subject'] = subject
    
        # Add the message body
        msg.attach(MIMEText(message, 'plain'))
    
        # Set up the server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Start TLS encryption
        server.login(from_email, password)  # Log in to your email account
    
        # Send the email to all recipients
        server.sendmail(from_email, recipients, msg.as_string())
    
        # Close the server connection
        server.quit()
    
        print(f"Email sent to: {', '.join(recipients)}")
    
    # Example Usage
    subject = "Pets Available Notification"
    message = "Pets are available on the following routes and dates."
    
    # List of email recipients
    recipients = ["beaquel13@gmail.com", "philip.kitchener1989@gmail.com"]  # Add as many emails as you need
    
    # Send the email
    send_email_alert(subject, message, recipients)
    
else:
    print('none found at this time')
    
# Delete PDFs after processing
delete_pdfs_from_folder(download_dir)


Cookies accepted.
Cookie overlay has disappeared.
Overlay removed using JavaScript.
Chevron icon clicked.
Overlay appeared.
5th option selected via JavaScript.
Button clicked successfully.
Error clicking the download button: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Cookies accepted.
Cookie overlay has disappeared.
Overlay removed using JavaScript.
Chevron icon clicked.
Overlay appeared.
6th option selected via JavaScript.
Button clicked successfully.
Error clicking the download button: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mj

In [121]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


def send_email_alert(subject, message, recipients):
        from_email = "philip.kitchener1989@gmail.com"  # Your email address
        password = "qlfwzcrviuczfujj"  # Your email password
    
        # Create the email message
        msg = MIMEMultipart()
        msg['From'] = from_email
        msg['To'] = ", ".join(recipients)  # Join the recipients' emails with commas
        msg['Subject'] = subject
    
        # Add the message body
        msg.attach(MIMEText(message, 'plain'))
    
        # Set up the server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Start TLS encryption
        server.login(from_email, password)  # Log in to your email account
    
        # Send the email to all recipients
        server.sendmail(from_email, recipients, msg.as_string())
    
        # Close the server connection
        server.quit()
    
        print(f"Email sent to: {', '.join(recipients)}")
    
# Example Usage
subject = "Test"
message = "Test."

# List of email recipients
recipients = ["beaquel13@gmail.com", "philip.kitchener1989@gmail.com"]  # Add as many emails as you need

# Send the email
send_email_alert(subject, message, recipients)

Email sent to: beaquel13@gmail.com, philip.kitchener1989@gmail.com
